In [51]:
pip install fuzzywuzzy

Note: you may need to restart the kernel to use updated packages.


In [52]:
import pandas as pd
import numpy as np
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import datetime

# Merging House and Senate Stocks

In [53]:
house = pd.read_csv('../data/house_stocks.csv')
senate = pd.read_csv('../data/senate_stocks.csv')

In [54]:
#senate['transaction_date'] = pd.to_datetime(senate['transaction_date'])
senate.drop(columns = ['asset_type', 'comment', 'ptr_link'], inplace=True)
senate['representative'] = senate['senator']
senate.drop(columns = 'senator', inplace = True)
senate.loc[senate['type'] == 'Purchase', 'type'] = 'purchase'
senate.loc[senate['type'] == 'Sale (Full)', 'type'] = 'sale_full'
senate.loc[senate['type'] == 'Sale (Partial)', 'type'] = 'sale_partial'
senate.loc[senate['type'] == 'Exchange', 'type'] = 'exchange'
senate['timestamp'] = [int(datetime.datetime.strptime(timer,"%m/%d/%Y").timestamp()) for 
                       timer in senate['transaction_date']]
senate['transaction_date'] = [str(datetime.datetime.strptime(timer,"%m/%d/%Y"))[:10] for 
                              timer in senate['transaction_date']]

In [55]:
#house['transaction_date'] = pd.to_datetime(house['transaction_date'])
house.drop(columns = ['disclosure_year', 'disclosure_date', 'district', 
                      'ptr_link', 'cap_gains_over_200_usd'], inplace = True)

house['representative'] = [rep[5:] for rep in house['representative']]
house['timestamp'] = [int(datetime.datetime.strptime(timer,"%Y-%m-%d").timestamp()) for 
                      timer in house['transaction_date']]

In [56]:
df = pd.concat([house, senate])
df.drop(columns = ['owner'], inplace = True)

In [57]:
df.dropna(inplace=True)
df.drop(df[df.ticker.str.contains('-')].index, inplace = True)
df.drop(df[df.ticker.str.contains('\.')].index, inplace = True)

In [59]:
df.to_csv('../data/senate_and_house.csv')

# Merging member years and main dataframe

In [60]:
members = pd.read_csv('../data/cleaned_members.csv')

In [61]:
main = pd.read_csv('../data/senate_and_house.csv')

In [62]:
members.head()

,people,state,party,total_senate_years,total_house_years,total_years
0,James Abdnor,South Dakota,Republican,6,8,14
1,Neil Abercrombie,Hawaii,Democratic,0,22,22
2,James Abourezk,South Dakota,Democratic,6,2,8
3,"Ralph Abraham, Lee",Louisiana,Republican,0,6,6
4,Spencer Abraham,Michigan,Republican,6,0,6


In [63]:
main.head()

,Unnamed: 0,transaction_date,ticker,asset_description,type,amount,representative,timestamp
0,4,2020-03-18,CBRE,CBRE Group Inc Common Stock Class A,sale_partial,"$1,001 - $15,000",Susie Lee,1584504000
1,5,2020-03-24,CNC,Centene Corporation,purchase,"$1,001 - $15,000",Susie Lee,1585022400
2,6,2020-03-31,DG,Dollar General Corporation,purchase,"$1,001 - $15,000",Susie Lee,1585627200
3,10,2020-02-21,ENV,"Envestnet, Inc",sale_partial,"$1,001 - $15,000",Susie Lee,1582261200
4,12,2020-03-17,LKQ,LKQ Corporation,sale_partial,"$1,001 - $15,000",Susie Lee,1584417600


In [64]:
# unique in main
names_in_main = main.representative.unique()

# made into a list 
names = []
for name in names_in_main:
    names.append(name)
type(names)

#list of members df 
member_names = []
for name in members.people.values:
    member_names.append(name)

In [65]:
len(member_names)

2422

In [66]:
# unique names in main
keys = {}

for name in names:
    #finding closest math to main data from larger 'members data'
    keys[name] = ((process.extract(name, member_names, limit=2)))

In [67]:
keys

{'Susie Lee': [('Susie Lee', 100), ('Ralph Abraham, Lee', 86)],
 'Alan S. Lowenthal': [('Alan Lowenthal, S.', 95), ('Alan Bible', 86)],
 'Mo Brooks': [('Mo Brooks', 100), ('Jack Brooks, B.', 86)],
 'Robert J. Wittman': [('Robert Wittman, J.', 95), ('J. Beall', 86)],
 'Lois Frankel': [('Lois Frankel', 100), ('Al Franken', 73)],
 'Michael T. McCaul': [('Michael McCaul, T.', 95),
  ('Michael McNulty, R.', 75)],
 'Suzan K. DelBene': [('Suzan DelBene, K.', 95), ('John Delaney, K.', 59)],
 'Greg Gianforte': [('Greg Gianforte', 100), ('Greg Ganske', 72)],
 'Lloyd K. Smucker': [('Lloyd Smucker', 95), ('Floyd Haskell, K.', 64)],
 'Earl Blumenauer': [('Earl Blumenauer', 100), ('Earl Hutto', 86)],
 'James Comer': [('James Comer', 100), ('James Broyhill, T.', 86)],
 'James R. Langevin': [('James Langevin, R.', 95), ('James Comer', 86)],
 'Anthony E. Gonzalez': [('Anthony Gonzalez', 95), ('E. Garn', 86)],
 'Josh Gottheimer': [('Josh Gottheimer', 100), ('Joshua Eilberg', 62)],
 'Katherine M. Clark':

In [68]:
#where scores live
# for i in range(1, 173):
#     print(keys[i][1][1])

In [69]:
merge_key = [keys[name][0][0] for name in main['representative']]

In [70]:
main['merge_key'] = merge_key

In [71]:
main = main.merge(members, how = 'left', left_on = 'merge_key', right_on='people')

In [72]:
df = main

# Merge Donations with Main

In [73]:
donations = pd.read_csv('../data/congress_donations.csv')
donations['party'] = [name[-2] for name in donations['Name']]
donations.columns = donations.columns.str.lower().str.replace(' ','_')
donations['name'] = [name[:-4] for name in donations['name']]

In [74]:
df.head()

,Unnamed: 0,transaction_date,ticker,asset_description,type,amount,representative,timestamp,merge_key,people,state,party,total_senate_years,total_house_years,total_years
0,4,2020-03-18,CBRE,CBRE Group Inc Common Stock Class A,sale_partial,"$1,001 - $15,000",Susie Lee,1584504000,Susie Lee,Susie Lee,Nevada,Democratic,0,2,2
1,5,2020-03-24,CNC,Centene Corporation,purchase,"$1,001 - $15,000",Susie Lee,1585022400,Susie Lee,Susie Lee,Nevada,Democratic,0,2,2
2,6,2020-03-31,DG,Dollar General Corporation,purchase,"$1,001 - $15,000",Susie Lee,1585627200,Susie Lee,Susie Lee,Nevada,Democratic,0,2,2
3,10,2020-02-21,ENV,"Envestnet, Inc",sale_partial,"$1,001 - $15,000",Susie Lee,1582261200,Susie Lee,Susie Lee,Nevada,Democratic,0,2,2
4,12,2020-03-17,LKQ,LKQ Corporation,sale_partial,"$1,001 - $15,000",Susie Lee,1584417600,Susie Lee,Susie Lee,Nevada,Democratic,0,2,2


In [75]:
donations.head()

,name,total_raised,total_from_small_donors,percent_from_small_donors*,total_money_raised,party
0,Ocasio-CortezAlexandria Ocasio-Cortez,$20664795,$16434594,79.53%,Raised over $100k,D
1,TrahanLori Trahan,$1449303,$1151465,79.45%,Raised over $100k,D
2,SandersBernie Sanders,$21565004,$15151822,70.26%,Raised over $100k,D
3,HunterDuncan D. Hunter,$1693623,$1142274,67.45%,Raised over $100k,R
4,JordanJim Jordan,$18313823,$12122206,66.19%,Raised over $100k,R


In [76]:
def clean_name(name):
    first = name.split()[0]
    second = name.split()[1:]
    first_rev = first[::-1]
    for i, let in enumerate(first_rev):
        if let == let.upper():
            index = i
            break
    for word in second:
        if len(word) < 3 or '\.' in word:
            second.remove(word)
    first = first[-(i+1):]
    second.insert(0,first)
    new = ' '.join(second)
    return new

In [77]:
donations['name'] = [clean_name(name) for name in donations['name']]

In [78]:
len(donations.name.unique())

537

In [79]:
donations.shape

(537, 6)

In [80]:
len(df.merge_key.unique())

166

In [81]:
df.columns

Index(['Unnamed: 0', 'transaction_date', 'ticker', 'asset_description', 'type',
       'amount', 'representative', 'timestamp', 'merge_key', 'people', 'state',
       'party', 'total_senate_years', 'total_house_years', 'total_years'],
      dtype='object')

In [82]:
# get unique list of names in main
names_in_df = df.representative.unique()
# make into a list 
names = []
for name in names_in_df:
    names.append(name)
type(names)
#list of members df 
member_names = []
for name in donations.name.values:
    member_names.append(name)

In [83]:
keys = {}
for name in names:  #names in smaller dataset to use as merge key
    #finding closest math to main data from larger 'members data'
    keys[name] = ((process.extract(name, member_names, limit=2))) 
    #you can limit to 1 to take less time but I wanted to check

In [84]:
#checking names that might not match correctly
for name in keys:
    if keys[name][0][1] <= 86:
        print(name, keys[name])

Robert J. Wittman [('J Cox', 86), ('Rob Wittman', 86)]
James R. Langevin [('James Risch', 86), ('James Comer', 86)]
Gerald E. Connolly [('Gerry Connolly', 75), ('J Cox', 54)]
Kathy Manning [('Kay Granger', 58), ('Kathy Castor', 56)]
Michael K. Simpson [('Mike Simpson', 86), ('Michael Waltz', 66)]
Rohit Khanna [('Ro Khanna', 86), ('Lori Trahan', 52)]
Christopher L. Jacobs [('Chris Jacobs', 86), ('Christopher Murphy', 72)]
eter Meijer [('Peter DeFazio', 58), ('Peter Welch', 55)]
James E Hon Banks [('James Risch', 86), ('Jim Banks', 86)]
Neal Patrick MD, Facs Dunn [('Patrick Leahy', 86), ('Neal Dunn', 86)]
Sara Jacobs [('Chris Jacobs', 70), ('Jose Serrano', 58)]
Marie Newman [('Madeleine Dean', 62), ('Mike Conaway', 58)]
Gilbert Cisneros [('Gil Cisneros', 86), ('Bernie Sanders', 60)]
K. Michael Conaway [('Mike Conaway', 86), ('Michael Waltz', 66)]
Neal Patrick MD, FACS Dunn [('Patrick Leahy', 86), ('Neal Dunn', 86)]
James E. Banks [('Jim Banks', 86), ('James Lankford', 64)]
Donald Sternof

In [85]:
no_match = ['Michael Enzi', 'Kathy Manning', 'John Hickenlooper', 'William Hagerty', 'cott Franklin', 'Victoria Spartz',
           'Marjorie Taylor Greene', 'Marie Newman', 'eter Meijer', 'David Cheston Rouzer', 'Robert "Bobby" Scott']

second_match = ['Mitchell Mcconnell, Jr.', 'Harold Dallas Rogers', 'Neal Patrick Dunn MD, FACS']

In [86]:
keys[name][1][0]

'Cory Gardner'

In [87]:
donation_merge_key = [keys[name][1][0] if
                      name in second_match \
                      else None if name in no_match else
                      keys[name][0][0] for name in main['representative']]

In [88]:
donation_merge_key

['Susie Lee',
 'Susie Lee',
 'Susie Lee',
 'Susie Lee',
 'Susie Lee',
 'Susie Lee',
 'Susie Lee',
 'Susie Lee',
 'Susie Lee',
 'Susie Lee',
 'Susie Lee',
 'Susie Lee',
 'Susie Lee',
 'Susie Lee',
 'Alan Lowenthal',
 'Alan Lowenthal',
 'Alan Lowenthal',
 'Alan Lowenthal',
 'Alan Lowenthal',
 'Alan Lowenthal',
 'Alan Lowenthal',
 'Alan Lowenthal',
 'Alan Lowenthal',
 'Mo Brooks',
 'Mo Brooks',
 'Alan Lowenthal',
 'Alan Lowenthal',
 'Alan Lowenthal',
 'Alan Lowenthal',
 'Alan Lowenthal',
 'Alan Lowenthal',
 'J Cox',
 'J Cox',
 'J Cox',
 'J Cox',
 'J Cox',
 'Lois Frankel',
 'Lois Frankel',
 'Lois Frankel',
 'Michael McCaul',
 'Michael McCaul',
 'Michael McCaul',
 'Michael McCaul',
 'Michael McCaul',
 'Michael McCaul',
 'Michael McCaul',
 'Michael McCaul',
 'Michael McCaul',
 'Michael McCaul',
 'Michael McCaul',
 'Michael McCaul',
 'Michael McCaul',
 'Michael McCaul',
 'Michael McCaul',
 'Michael McCaul',
 'Michael McCaul',
 'Michael McCaul',
 'Michael McCaul',
 'Michael McCaul',
 'Michael 

In [89]:
df['donation_merge_key'] = donation_merge_key

In [90]:
df.head()

,Unnamed: 0,transaction_date,ticker,asset_description,type,amount,representative,timestamp,merge_key,people,state,party,total_senate_years,total_house_years,total_years,donation_merge_key
0,4,2020-03-18,CBRE,CBRE Group Inc Common Stock Class A,sale_partial,"$1,001 - $15,000",Susie Lee,1584504000,Susie Lee,Susie Lee,Nevada,Democratic,0,2,2,Susie Lee
1,5,2020-03-24,CNC,Centene Corporation,purchase,"$1,001 - $15,000",Susie Lee,1585022400,Susie Lee,Susie Lee,Nevada,Democratic,0,2,2,Susie Lee
2,6,2020-03-31,DG,Dollar General Corporation,purchase,"$1,001 - $15,000",Susie Lee,1585627200,Susie Lee,Susie Lee,Nevada,Democratic,0,2,2,Susie Lee
3,10,2020-02-21,ENV,"Envestnet, Inc",sale_partial,"$1,001 - $15,000",Susie Lee,1582261200,Susie Lee,Susie Lee,Nevada,Democratic,0,2,2,Susie Lee
4,12,2020-03-17,LKQ,LKQ Corporation,sale_partial,"$1,001 - $15,000",Susie Lee,1584417600,Susie Lee,Susie Lee,Nevada,Democratic,0,2,2,Susie Lee


In [91]:
df = df.merge(donations, how = 'left', left_on = 'donation_merge_key', right_on='name')

In [92]:
df.columns

Index(['Unnamed: 0', 'transaction_date', 'ticker', 'asset_description', 'type',
       'amount', 'representative', 'timestamp', 'merge_key', 'people', 'state',
       'party_x', 'total_senate_years', 'total_house_years', 'total_years',
       'donation_merge_key', 'name', 'total_raised', 'total_from_small_donors',
       'percent_from_small_donors*', 'total_money_raised', 'party_y'],
      dtype='object')

In [93]:
df.drop(columns = ['Unnamed: 0'], inplace = True)

In [94]:
df.columns

Index(['transaction_date', 'ticker', 'asset_description', 'type', 'amount',
       'representative', 'timestamp', 'merge_key', 'people', 'state',
       'party_x', 'total_senate_years', 'total_house_years', 'total_years',
       'donation_merge_key', 'name', 'total_raised', 'total_from_small_donors',
       'percent_from_small_donors*', 'total_money_raised', 'party_y'],
      dtype='object')

In [95]:
df.isna().sum()

transaction_date                0
ticker                          0
asset_description               0
type                            0
amount                          0
representative                  0
timestamp                       0
merge_key                       0
people                          0
state                           0
party_x                         0
total_senate_years              0
total_house_years               0
total_years                     0
donation_merge_key            448
name                          448
total_raised                  448
total_from_small_donors       448
percent_from_small_donors*    448
total_money_raised            448
party_y                       448
dtype: int64

In [96]:
df.to_csv('../data/house_senate_years_donations.csv', index = False)